In [137]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [138]:
doc_1 = "DBLog est un framework générique de capture de données de changement qui permet de capturer les modifications validées à partir d'une base de données en temps réel et de propager ces modifications aux consommateurs de downstream. Les journaux de transactions des bases de données sont la source des événements CDC. DBLog est un framework basé sur Java, capable de capturer les changements en temps réel et de faire des dumps. Nous utilisons Zookeeper pour stocker l'état lié au traitement des journaux et des dumps, et pour l'élection du leader. Pour utiliser DBLog, une base de données doit fournir un journal des modifications à partir d'un historique linéaire des modifications validées et ces conditions sont remplies par des systèmes tels que MySQL, PostgreSQL, MariaDB, etc. Le traitement de vidage a été intégré en utilisant SQL et JDBC, ne nécessitant que la mise en œuvre de la sélection de blocs et la mise à jour du filigrane. Le même code est utilisé pour MySQL et PostgreSQL et peut également être utilisé pour d'autres bases de données similaires. DBLog utilise une architecture active-passive. Une instance est active et les autres sont en veille passive. Nous utilisons Zookeeper pour l'élection du leader afin de déterminer l'instance active."
#Resp 65 e 67:
doc_2 = "On a plusieurs bases de données (probablement distribuées), qui produisent des logs. Ces logs rentrent dans un service Apache Kafka."
doc_3 = "Je comprends que des données passent d'une source vers un log vers le sink. Je vois également les différentes BD utilisées. "

corpus = np.array([doc_1, doc_2, doc_3])

### 1) Aplicar lowercase e remover caracteres

In [139]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [140]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['espresso is linkedins documentoriented timelineconsistent distributed data store clients fetching profile data read subset of fields of whole documents from espresso in different use cases there are three main parts of espresso routers storage nodes and cluster management routers receive requests from online clients and forward the requests to storage nodes routes also inspect uri to get the database name and look up the routing table for appropriate storage nodes to send requests to it also assembles the results and sends it back to clients storage node stores and serves documents provides transactions support for documents with the same partition key and maintains commit logs for replication espresso uses apache helix to monitor and maintain the cluster'
 'can understand that there are several clients that communicate with routers just as there are several routers that communicate with storages nodes where each node contains database there is set of relays databus that communicate 

### 2) Remover stopwords

In [141]:
from nltk.corpus import stopwords
def clear_stopwords(text, language):
    filtered_text = []
    for t in text:
        words = [w for w in t.split() if not w in stopwords.words(language)]
        words = ' '.join(words)
        filtered_text.append(words)
    return np.array(filtered_text)   

In [143]:
corpus_filtered = clear_stopwords(corpus_clear, "french")
print(corpus_filtered)

['espresso is linkedins documentoriented timelineconsistent distributed data store clients fetching profile data read subset of fields of whole documents from espresso in different use cases there are three main parts of espresso routers storage nodes and cluster management routers receive requests from online clients and forward the requests to storage nodes routes also inspect uri to get the database name and look up the routing table appropriate storage nodes to send requests to it also assembles the results and sends it back to clients storage node stores and serves documents provides transactions support documents with the same partition key and maintains commit logs replication espresso uses apache helix to monitor and maintain the cluster'
 'can understand that there are several clients that communicate with routers just there are several routers that communicate with storages nodes where each node contains database there is set of relays databus that communicate the storages al

### Usando a biblioteca sklearn

In [144]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [145]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorize = CountVectorizer()
vector_matrix = vectorize.fit_transform(corpus_filtered)

tokens = vectorize.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)



,accessing,all,also,and,apache,appropriate,are,assembles,back,between,...,transactions,understand,up,uri,use,uses,well,where,whole,with
text_1,0,0,2,7,1,1,1,1,1,0,...,1,0,1,1,1,1,0,0,1,1
text_2,0,2,0,0,0,0,5,0,0,0,...,0,1,0,0,0,0,0,1,0,2
text_3,1,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
text_4,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [146]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['text_1','text_2','text_3'])

,text_1,text_2,text_3,text_4
text_1,1.000000,0.246008,0.431661,0.395190
text_2,0.246008,1.000000,0.422036,0.289496
text_3,0.431661,0.422036,1.000000,0.251964
text_4,0.395190,0.289496,0.251964,1.000000
